In [104]:
import torch
import torch.nn as nn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import arkad
from arkad import BaseModel
from torch.utils.data import Dataset, DataLoader
from pydantic import BaseModel as PydanticBaseModel

import sys
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer

current_dir = os.getcwd()

sys.path.append(os.path.abspath(os.path.join(current_dir, '..')))

from src.datasets import *
from src.models import *
from src.train_test import *

In [105]:
# Defina os hiperparâmetros
hidden_dim = [768, 512, 256]
encoding_size = 128
learning_rate = 3e-4
num_epochs = 15
batch_size = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recrie a instância do modelo (use os mesmos hiperparâmetros)
model_loaded = VAE(
    hidden_dim=hidden_dim,
    encoding_size=encoding_size,
    output_dim_classifier=7,
    dropout=0.5,
    batch_norm_1d=True
)

model_loaded.load_state_dict(torch.load('../models/vae_model_state_dict_2.pth'))

model_loaded = model_loaded.to('cpu')

# get csv 
data = pd.read_csv('../data/data.csv')

# Load vector data
mu_outputs_ = pd.read_csv('../data/mu_outputs.csv')

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

C:\Users\gabri\AppData\Local\Temp\ipykernel_62920\1735819938.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_loaded.load_state_dict(torch.load('../models/vae_mode

In [106]:
def invoke(query, k=10):
    model_loaded.eval()
    query_processed = tokenizer(query, return_tensors='pt', padding=True, truncation=True)
    query_embedding_mu = model_loaded.encoder(input_ids=query_processed['input_ids'].to('cpu'), 
                                                attention_mask=query_processed['attention_mask'].to('cpu'))[0]
    
    query_embedding_mu = query_embedding_mu.cpu().detach().numpy()[0]
    query_embedding_mu = query_embedding_mu.reshape(1, -1)
    
    scores = cosine_similarity(mu_outputs_, query_embedding_mu)
    idxs = np.argsort(scores, axis=0)[::-1][:k]
    idxs_and_scores = np.array([[idx[0], scores[idx[0]][0]] for idx in idxs])
    
    output_content = data.iloc[idxs_and_scores[:, 0]]['text'].values
    output_url = data.iloc[idxs_and_scores[:, 0]]['url'].values
    output_scores = idxs_and_scores[:, 1]
    
    return output_content, output_url, output_scores

In [107]:
invoke('No recorte do futebol, sabemos o maracanã ficará lotado no jogo do palmeiras', k=10)

(array(['Site de acompanhantes, Fatal Model patrocinará o Vila Nova-GO\nAcordo inclui exposição na camisa e a instalação de telão no estádio; empresa também patrocina o Vitória-BA e a Ponte Preta-SP\nO Vila Novaanunciounesta 2ª feira (16.set.2024) um acordo de patrocínio com a Fatal Model, um site brasileiro de acompanhantes. A parceria inclui a presença da marca na barra frontal da camisa do time e a instalação de um novo telão no estádio OBA (Onésio Brasileiro Alvarenga). Embora o valor do contrato não tenha sido divulgado, ele é o maior já negociado para a área do uniforme do clube goiano.\nAlém da exposição na camisa e no estádio, o acordo estabelece espaços publicitários nosbackdropsde entrevistas a jornalistas. A Fatal Model também terá a oportunidade de realizar postagens em conjunto com o clube nas redes sociais.\nMurilo Reis, diretor de marketing do Vila Nova, falou sobre a importância da parceria.“É muito importante para o Vila Nova estabelecer uma parceria com uma empresa do

In [ ]:
from huggingface_hub import hf_hub_download
from urllib.parse import urlparse

class Output(PydanticBaseModel):
    content: str
    url: str
    score: float

class Retriever:
    def __init__(self, 
                 path_saved_model='https://huggingface.co/xValentim/vector-search-bert-based/resolve/main/vae_model_state_dict_2.pth', 
                 sample_size=None, 
                 path_saved_index='./data/mu_outputs.csv'):
        
        # Defina os hiperparâmetros
        hidden_dim = [768, 512, 256]
        encoding_size = 128
        learning_rate = 3e-4
        num_epochs = 15
        batch_size = 64
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Recrie a instância do modelo (use os mesmos hiperparâmetros)
        model_loaded = VAE(
            hidden_dim=hidden_dim,
            encoding_size=encoding_size,
            output_dim_classifier=7,
            dropout=0.5,
            batch_norm_1d=True
        )
        
        # Check if the path_saved_model is a URL
        if path_saved_model.startswith('http'):
            # Parse the URL to get the repository ID and filename
            parsed_url = urlparse(path_saved_model)
            path_parts = parsed_url.path.split('/')
            repo_id = '/'.join(path_parts[1:3])  # e.g., 'xValentim/vector-search-bert-based'
            filename = path_parts[-1]  # e.g., 'vae_model_state_dict_2.pth'

            # Download the model file from the Hugging Face Hub
            state_dict_path = hf_hub_download(repo_id=repo_id, filename=filename)

            # Load the state dictionary
            state_dict = torch.load(state_dict_path, map_location='cpu')
        else:
            # Load from local path
            state_dict = torch.load(path_saved_model, map_location='cpu')

        model_loaded.load_state_dict(state_dict)
        model_loaded.to('cpu')
        
        self.model = model_loaded
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
        
        self.data = pd.read_csv('./data/data.csv')
        if sample_size:
            self.data = self.data.sample(sample_size)
        
        if path_saved_index is None:
            
            self.tokenized_data = tokenizer(self.data['text'].tolist(), padding=True, truncation=True, return_tensors='pt')
            self.dataset = TensorDataset(self.tokenized_data['input_ids'], self.tokenized_data['attention_mask'])
            self.dataloader = DataLoader(self.dataset, batch_size=128)  # Ajuste o batch_size conforme a capacidade da GPU
            
            mu_outputs = []
            model_loaded.eval()  # Colocar o modelo em modo de avaliação
            model_loaded.to(device)  # Mova o modelo para a cuda

            with torch.no_grad():  # Desativar o cálculo de gradiente para economizar memória
                for batch in tqdm(self.dataloader):
                    input_ids, attention_mask = batch
                    input_ids = input_ids.to(device)
                    attention_mask = attention_mask.to(device)
                    outputs = model_loaded.encoder(input_ids=input_ids, attention_mask=attention_mask)
                    mu_outputs.append(outputs[0])

            # Concatenar os batches para ter o resultado final
            self.mu_outputs = torch.cat(mu_outputs, dim=0).cpu().detach().numpy()
        else:
            self.mu_outputs = pd.read_csv(path_saved_index)
            self.mu_outputs = self.mu_outputs.values
        
    def invoke(self, query, k=10):
        self.model.eval()
        query_processed = self.tokenizer(query, return_tensors='pt', padding=True, truncation=True)
        query_embedding_mu = self.model.encoder(input_ids=query_processed['input_ids'].to('cpu'), 
                                                  attention_mask=query_processed['attention_mask'].to('cpu'))[0]
        query_embedding_mu = query_embedding_mu.cpu().detach().numpy()[0]
        query_embedding_mu = query_embedding_mu.reshape(1, -1)
        
        scores = cosine_similarity(self.mu_outputs, query_embedding_mu)
        idxs = np.argsort(scores, axis=0)[::-1][:k]
        idxs_and_scores = np.array([[idx[0], scores[idx[0]][0]] for idx in idxs])
        
        output_content = self.data.iloc[idxs_and_scores[:, 0]]['text'].values
        output_url = self.data.iloc[idxs_and_scores[:, 0]]['url'].values
        output_scores = idxs_and_scores[:, 1]
        
        return output_content, output_url, output_scores
    
    def query(self, query, k=3):
        output_content, output_url, output_scores = self.invoke(query, k)
        output = [Output(content=output_content[i],
                         url=output_url[i],
                         score=output_scores[i]
                    ) for i in range(k)]
        return output

In [ ]:
retriever = Retriever(path_saved_index='../data/mu_outputs.csv',
                      path_saved_model='https://huggingface.co/xValentim/vector-search-bert-based/resolve/main/vae_model_state_dict_2.pth')

In [110]:
response = retriever.query('Nas últimas semanas a economia tem ido bastante mal durante o periodo das eleições, o dolar despencou', 
                           k=3)

In [111]:
response

[Output(content='OCDE projeta crescimento global de 3,2% em 2024 e 2025\nQueda da inflação e política monetária menos restritiva nos países contribuirão para os resultados\nAOCDE(Organização para Cooperação e Desenvolvimento Econômico) divulgou nesta 4ª feira (25.set.2024) estimativas atualizadas para o crescimento global. A organização projeta uma aceleração de 3,2% na economia em 2024, aumentando a estimativa anterior (3,1%) em 0,1 ponto percentual.\nA projeção para 2025 também é de 3,2%. O crescimento em ambos os anos deve se dar“com uma maior desaceleração da inflação, melhora da renda real e políticas monetárias menos restritivas em muitas economias ajudando a sustentar a demanda”. Eis aíntegrado relatório (PDF – 1 MB, em inglês).\nSegundo a OCDE, o crescimento econômico do G20“deve permanecer amplamente estável, apesar dos diferentes resultados projetados dentro do grupo”. A estimativa de aceleração do PIB em 2024 é de 3,2%. Deve desacelerar para 3,1% em 2025.\nA organização tamb